# QLoRA Fine-tuning + Evaluation (ClassicModels NL→SQL)

This notebook is the fine-tuning arm of the replication study. It adapts the base model with QLoRA on non-overlapping train data, then evaluates with the same `nl2sql.eval.eval_run` harness used for baseline prompting so weight adaptation effects are isolated.

Replication role:
- compare base vs fine-tuned at matched prompt settings (`k=0`, `k=3`),
- test whether fine-tuning trends reported by Ojuri et al. (2025) hold in an open-source local setup.

## Expected inputs
- Test set (fixed): `data/classicmodels_test_200.json`
- Training set (you create): `data/train/classicmodels_train_200.jsonl` (JSON Lines with `nlq` + `sql` per row)

## Outputs
- Adapter checkpoint: `results/adapters/qlora_classicmodels/`
- Eval outputs: `results/qlora/results_*_200.json`

Note: `results/` is gitignored by default. Download the outputs from Colab when finished.


## One-time setup (run first in a fresh Colab GPU runtime)
Run this cell as the first step in a fresh runtime. Let it finish, then **Runtime → Restart runtime** once, and run the rest of the notebook top-to-bottom. This pins torch/bitsandbytes/triton to CUDA 12.1 so 4-bit loading works.

Setup note: pinned versions reduce Colab drift and keep QLoRA runs repeatable.


In [ ]:

%%bash
set -e
export PIP_DEFAULT_TIMEOUT=120

# Clean conflicting preinstalls
pip uninstall -y torch torchvision torchaudio bitsandbytes triton transformers accelerate peft trl datasets numpy pandas fsspec requests google-auth || true

# Base deps
pip install -q --no-cache-dir --force-reinstall   numpy==1.26.4 pandas==2.2.1 fsspec==2024.5.0 requests==2.31.0 google-auth==2.43.0

# Torch + CUDA 12.1
pip install -q --no-cache-dir --force-reinstall   torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121   --index-url https://download.pytorch.org/whl/cu121

# bitsandbytes + triton + HF stack
pip install -q --no-cache-dir --force-reinstall   bitsandbytes==0.43.3 triton==2.3.1   transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0

echo "Setup complete. Restart runtime once, then run the rest of the notebook."


Prompt/eval: build prompts (system+schema+k exemplars), generate SQL, postprocess, and compute VA/EX/EM.

Prompting note: evaluation prompt structure matches baseline so fine-tuning effects are isolated.


In [ ]:
import os, sys, shutil
from pathlib import Path

# If opened directly in Colab, clone the repo first
if Path("data/classicmodels_test_200.json").exists() is False and Path("/content").exists():
    repo_dir = Path("/content/NLtoSQL")
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git "{repo_dir}"
    os.chdir(repo_dir)

sys.path.insert(0, os.getcwd())
print("cwd:", os.getcwd())


## 0) Install dependencies (Colab)

Install pinned dependencies from `requirements.txt`. Colab often needs a **runtime restart** after installs (Runtime → Restart runtime), then rerun from the top.


In [ ]:
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -r requirements.txt

    import torch
    import accelerate
    import peft
    import transformers
    import trl

    print('torch:', torch.__version__, 'cuda:', torch.cuda.is_available())
    print('transformers:', transformers.__version__)
    print('accelerate:', accelerate.__version__)
    print('peft:', peft.__version__)
    print('trl:', trl.__version__)

    if not torch.cuda.is_available():
        print('WARNING: CUDA is not available. In Colab, use a GPU runtime and avoid installing CPU-only torch wheels.')
        print('If you just changed torch packages, do: Runtime -> Restart runtime, then run from the top.')
else:
    print('Not in Colab; ensure requirements are installed.')


## 1) Authentication (GCP + Hugging Face)

- GCP auth is required for Cloud SQL access (VA evaluation).
- HF auth is required for gated models (Meta Llama 3).


In [3]:
# GCP auth (Colab) — safe to skip locally if using ADC
try:
    from google.colab import auth
except ModuleNotFoundError:
    auth = None
if auth:
    auth.authenticate_user()
else:
    print("Not running in Colab; ensure ADC/service account auth is configured.")

# Hugging Face auth
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
    print("Using HF token from env")
else:
    try:
        from huggingface_hub import notebook_login
        notebook_login()
    except Exception as e:
        print("HF auth not configured:", e)


## 2) Load benchmark + training set

Training set must be separate from the 200-item benchmark.

Recommended workflow:
- Run `notebooks/04_build_training_set.ipynb` to validate (and edit if needed) `data/train/classicmodels_train_200.jsonl`.


Data note: schema-aware prompts are kept consistent with baseline for fair comparison.


In [ ]:
import json
from pathlib import Path

test_path = Path("data/classicmodels_test_200.json")
train_path = Path("data/train/classicmodels_train_200.jsonl")

test_set = json.loads(test_path.read_text(encoding="utf-8"))
print("Test items:", len(test_set))

if not train_path.exists():
    raise FileNotFoundError(
        f"Missing training set at {train_path}. Create it before running QLoRA. "
        "Expected JSONL lines with keys: nlq, sql."
    )

train_records = []
for line in train_path.read_text(encoding="utf-8").splitlines():
    line = line.strip()
    if not line:
        continue
    train_records.append(json.loads(line))

print("Train items:", len(train_records))


### Leakage check (train vs test)

At minimum, ensure there is no exact NLQ overlap.


In [ ]:
test_nlqs = {item["nlq"].strip() for item in test_set}
train_nlqs = [r.get("nlq", "").strip() for r in train_records]
overlap = sorted({nlq for nlq in train_nlqs if nlq in test_nlqs})

print("NLQ overlap count:", len(overlap))
if overlap:
    print("Example overlaps:")
    for x in overlap[:10]:
        print("-", x)
    raise ValueError("Training set overlaps test set; remove overlapping items before training.")


## 3) DB engine + schema summary

Schema grounding is kept consistent with the baseline by using `nl2sql.schema.build_schema_summary`.


Schema note: `nl2sql.schema` builds the same schema summary format used in baseline runs.


Environment note: DB connection path is identical to baseline (`nl2sql.db`).


In [ ]:
from getpass import getpass
from nl2sql.db import create_engine_with_connector
from nl2sql.schema import build_schema_summary

INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME", "classicmodels")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

engine, connector = create_engine_with_connector(
    instance_connection_name=INSTANCE_CONNECTION_NAME,
    user=DB_USER,
    password=DB_PASS,
    db_name=DB_NAME,
)

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME, max_cols_per_table=50)
print("Schema summary length:", len(SCHEMA_SUMMARY))


## 4) Load base model (4-bit) + configure QLoRA


Model note: the base model is loaded in 4-bit before attaching LoRA adapters.


Fallback note: if memory is tight, lower batch/accumulation settings rather than changing evaluation logic.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

if not torch.cuda.is_available():
    raise RuntimeError('CUDA is not available. In Colab, switch to a GPU runtime: Runtime -> Change runtime type -> GPU.')

cc_major, cc_minor = torch.cuda.get_device_capability(0)
use_bf16 = cc_major >= 8  
compute_dtype = torch.bfloat16 if use_bf16 else torch.float16

print('GPU:', torch.cuda.get_device_name(0))
print('Compute capability:', (cc_major, cc_minor))
print('Using bf16:', use_bf16, '| compute_dtype:', compute_dtype)

tok = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

# transformers/bitsandbytes 4-bit quantization does not allow some layers to be auto-offloaded
# to CPU/disk. Force the whole model onto GPU:0. If you OOM, restart runtime and close other
# notebooks/tabs, or use a higher-memory GPU (A100/L4).
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map=device_map,
    token=True,
)

# Deterministic defaults for later evaluation
base_model.generation_config.do_sample = False
base_model.generation_config.temperature = 1.0
base_model.generation_config.top_p = 1.0

base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


## 5) Build the SFT dataset


In [ ]:
from datasets import Dataset
from nl2sql.prompting import SYSTEM_INSTRUCTIONS

def format_example(nlq: str, sql: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_INSTRUCTIONS},
        {"role": "user", "content": "Schema:\n" + SCHEMA_SUMMARY},
        {"role": "user", "content": f"NLQ: {nlq}"},
        {"role": "assistant", "content": sql.rstrip(";") + ";"},
    ]
    return tok.apply_chat_template(messages, tokenize=False)

train_texts = [format_example(r["nlq"], r["sql"]) for r in train_records]
train_ds = Dataset.from_dict({"text": train_texts})
print(train_ds)


## 6) Train (SFT with TRL)


Training note: TRL `SFTTrainer` + PEFT LoRA provides the standard supervised QLoRA loop.


Stability note: keep train arguments and seed logged so runs are reproducible.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

output_dir = "results/adapters/qlora_classicmodels"

# T4 GPUs in Colab do not support bf16; use fp16 in that case.
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    num_train_epochs=3,
    warmup_ratio=0.05,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    bf16=use_bf16,
    fp16=(not use_bf16),
    optim="paged_adamw_8bit",
    report_to=[],
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tok,
    train_dataset=train_ds,
    dataset_text_field="text",
    args=training_args,
    max_seq_length=1024,
)

trainer.train()
trainer.model.save_pretrained(output_dir)
tok.save_pretrained(output_dir)
print("Saved adapters to:", output_dir)


## 7) Evaluate adapters on the same 200-item test set


Evaluation note: adapters are tested on the same fixed 200-item benchmark as baseline.


**Seed note (important):**
- A `seed` controls exemplar sampling when `k > 0`.
- Different seeds mean different few-shot exemplar sets from the same pool.
- For `k=0` there are no exemplars, so seed has no practical effect.

**Experiment knobs in next cell:**
- `K_VALUES` + `SEEDS` for k/seed sweeps.
- `EXEMPLAR_STRATEGY` for exemplar-pool ablation (`all`, `brief_sql`, `join_heavy`, `agg_heavy`).
- `PROMPT_VARIANT` for prompt ablation (`default`, `schema_only_minimal`, `no_routing_hints`).
- `SCHEMA_VARIANT` for schema-context ablation (`full`, `first_80_lines`, `first_40_lines`).


In [ ]:
import re
from peft import PeftModel
from nl2sql.eval import eval_run
from pathlib import Path
from datetime import datetime, timezone
import subprocess
import shutil
import pandas as pd
import nl2sql.prompting as prompting_mod

DEFAULT_SYSTEM_INSTRUCTIONS = prompting_mod.SYSTEM_INSTRUCTIONS

PROMPT_VARIANTS = {
    "default": DEFAULT_SYSTEM_INSTRUCTIONS,
    "schema_only_minimal": """You are an expert data analyst writing MySQL queries.
Given the database schema and a natural language question, write a single SQL SELECT query.

Rules:
- Output ONLY SQL (no explanation, no markdown).
- Output exactly ONE statement, starting with SELECT.
- Use only tables/columns listed in the schema.
""",
    "no_routing_hints": DEFAULT_SYSTEM_INSTRUCTIONS.split("- Routing hints:")[0].rstrip(),
}

def schema_variant_text(schema_text: str, variant: str) -> str:
    lines = schema_text.splitlines()
    if variant == "full":
        return schema_text
    if variant == "first_80_lines":
        return "\n".join(lines[:80])
    if variant == "first_40_lines":
        return "\n".join(lines[:40])
    raise ValueError(f"Unknown SCHEMA_VARIANT: {variant}")

def exemplar_pool_for_strategy(items: list[dict], strategy: str) -> list[dict]:
    if strategy == "all":
        return list(items)

    def _sql(x):
        return str(x.get("sql", "")).strip()

    def _is_join(sql: str) -> bool:
        s = sql.lower()
        return " join " in f" {s} "

    def _is_agg(sql: str) -> bool:
        return bool(re.search(r"\b(sum|avg|count|min|max)\s*\(", sql.lower()))

    if strategy == "brief_sql":
        ranked = sorted(items, key=lambda x: len(_sql(x)))
        keep = max(50, int(0.4 * len(ranked)))
        pool = ranked[:keep]
    elif strategy == "join_heavy":
        pool = [x for x in items if _is_join(_sql(x))]
    elif strategy == "agg_heavy":
        pool = [x for x in items if _is_agg(_sql(x))]
    else:
        raise ValueError(f"Unknown EXEMPLAR_STRATEGY: {strategy}")

    return pool if len(pool) >= 10 else list(items)

eval_base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map=device_map,
    token=True,
)
eval_base.generation_config.do_sample = False
eval_base.generation_config.temperature = 1.0
eval_base.generation_config.top_p = 1.0

eval_model = PeftModel.from_pretrained(eval_base, output_dir)

try:
    commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"]).decode().strip()
except Exception:
    commit = "unknown"

run_metadata_base = {
    "commit": commit,
    "model_id": MODEL_ID,
    "method": "qlora",
    "adapter_dir": output_dir,
}

Path("results/qlora").mkdir(parents=True, exist_ok=True)

def run_qlora_grid(
    *,
    k_values: list[int],
    seeds: list[int],
    run_tag: str,
    prompt_variant: str,
    schema_variant: str,
    exemplar_strategy: str,
    limit: int | None = None,
    copy_canonical: bool = True,
):
    if not seeds:
        raise ValueError("Provide at least one seed")

    if prompt_variant not in PROMPT_VARIANTS:
        raise ValueError(f"Unknown PROMPT_VARIANT: {prompt_variant}")

    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%SZ")
    run_dir = Path("results/qlora/runs") / f"{run_tag}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    schema_used = schema_variant_text(SCHEMA_SUMMARY, schema_variant)
    exemplar_pool = exemplar_pool_for_strategy(test_set, exemplar_strategy)

    rows = []
    primary_seed = seeds[0]

    old_prompt = prompting_mod.SYSTEM_INSTRUCTIONS
    prompting_mod.SYSTEM_INSTRUCTIONS = PROMPT_VARIANTS[prompt_variant]

    try:
        for k in k_values:
            seed_list = [primary_seed] if k == 0 else seeds
            for seed in seed_list:
                save_path = run_dir / f"results_k{k}_seed{seed}.json"

                run_meta = dict(run_metadata_base)
                run_meta.update({
                    "run_tag": run_tag,
                    "k": k,
                    "seed": seed,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                })

                items = eval_run(
                    test_set=test_set,
                    exemplar_pool=exemplar_pool,
                    k=k,
                    limit=limit,
                    seed=seed,
                    engine=engine,
                    model=eval_model,
                    tokenizer=tok,
                    schema_summary=schema_used,
                    save_path=str(save_path),
                    run_metadata=run_meta,
                    avoid_exemplar_leakage=True,
                )

                n = len(items)
                va = sum(int(x.va) for x in items) / max(n, 1)
                em = sum(int(x.em) for x in items) / max(n, 1)
                ex = sum(int(x.ex) for x in items) / max(n, 1)

                rows.append({
                    "run_tag": run_tag,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                    "k": k,
                    "seed": seed,
                    "n": n,
                    "va_rate": va,
                    "em_rate": em,
                    "ex_rate": ex,
                    "json_path": str(save_path),
                })

                if copy_canonical and seed == primary_seed and k in {0, 3}:
                    target = (
                        Path("results/qlora/results_zero_shot_200.json")
                        if k == 0
                        else Path("results/qlora/results_few_shot_k3_200.json")
                    )
                    target.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(save_path, target)
                    print(f"Updated canonical file: {target}")
    finally:
        prompting_mod.SYSTEM_INSTRUCTIONS = old_prompt

    df = pd.DataFrame(rows).sort_values(["k", "seed"]).reset_index(drop=True)
    df.to_csv(run_dir / "grid_summary.csv", index=False)

    agg = (
        df.groupby(["prompt_variant", "schema_variant", "exemplar_strategy", "k"], as_index=False)
        .agg(
            runs=("seed", "count"),
            va_mean=("va_rate", "mean"),
            va_std=("va_rate", "std"),
            em_mean=("em_rate", "mean"),
            em_std=("em_rate", "std"),
            ex_mean=("ex_rate", "mean"),
            ex_std=("ex_rate", "std"),
        )
    )
    agg.to_csv(run_dir / "grid_summary_by_k.csv", index=False)

    print("Saved grid run to:", run_dir)
    return df, agg, run_dir

# ============================
# QUICK MODE (recommended default)
K_VALUES = [0, 3]
SEEDS = [7]
RUN_TAG = "qlora_main"
PROMPT_VARIANT = "default"
SCHEMA_VARIANT = "full"
EXEMPLAR_STRATEGY = "all"

# FULL K/S E1 SWEEP (uncomment for primary experiment)
# K_VALUES = [0, 1, 3, 5, 8]
# SEEDS = [7, 17, 27, 37, 47]
# RUN_TAG = "qlora_e1_k_sweep"
# PROMPT_VARIANT = "default"
# SCHEMA_VARIANT = "full"
# EXEMPLAR_STRATEGY = "all"

# EXEMPLAR STRATEGY ABLATION EXAMPLE (few-shot only; keep k>0 values)
# K_VALUES = [3]
# SEEDS = [7, 17, 27, 37, 47]
# RUN_TAG = "qlora_exemplar_brief"
# EXEMPLAR_STRATEGY = "brief_sql"

# PROMPT/SCHEMA ABLATION EXAMPLE
# K_VALUES = [0, 3]
# SEEDS = [7]
# RUN_TAG = "qlora_prompt_ablation_schema_only"
# PROMPT_VARIANT = "schema_only_minimal"
# SCHEMA_VARIANT = "first_80_lines"

qlora_grid, qlora_by_k, qlora_run_dir = run_qlora_grid(
    k_values=K_VALUES,
    seeds=SEEDS,
    run_tag=RUN_TAG,
    prompt_variant=PROMPT_VARIANT,
    schema_variant=SCHEMA_VARIANT,
    exemplar_strategy=EXEMPLAR_STRATEGY,
    limit=None,
)

print("\nPer-run rows:")
display(qlora_grid)
print("\nPer-k summary (mean/std across seeds):")
display(qlora_by_k)


## 8) Compare against baseline outputs (optional)


In [ ]:
import json
from pathlib import Path

baseline_zero = Path("results/baseline/results_zero_shot_200.json")
baseline_few  = Path("results/baseline/results_few_shot_k3_200.json")

if baseline_zero.exists() and baseline_few.exists():
    b0 = json.loads(baseline_zero.read_text(encoding="utf-8"))
    b3 = json.loads(baseline_few.read_text(encoding="utf-8"))
    q0 = json.loads(Path("results/qlora/results_zero_shot_200.json").read_text(encoding="utf-8"))
    q3 = json.loads(Path("results/qlora/results_few_shot_k3_200.json").read_text(encoding="utf-8"))

    print("Baseline zero-shot:", "VA", round(b0["va_rate"], 3), "EM", round(b0.get("em_rate", 0.0), 3), "EX", round(b0["ex_rate"], 3))
    print("QLoRA   zero-shot:", "VA", round(q0["va_rate"], 3), "EM", round(q0.get("em_rate", 0.0), 3), "EX", round(q0["ex_rate"], 3))
    print("Baseline few-shot :", "VA", round(b3["va_rate"], 3), "EM", round(b3.get("em_rate", 0.0), 3), "EX", round(b3["ex_rate"], 3))
    print("QLoRA   few-shot :", "VA", round(q3["va_rate"], 3), "EM", round(q3.get("em_rate", 0.0), 3), "EX", round(q3["ex_rate"], 3))
else:
    print("Baseline JSONs not found under results/baseline/. Run the baseline notebook first (or upload the JSONs).")
